In [12]:
import scipy.io as spio
import matplotlib.pyplot as plt
import numpy as np
import numpy_indexed as npi
import pandas as pd
import os
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
import warnings
warnings.filterwarnings(action='once')

forcing_file = '../data/raw/forcing_data/1750-Oct2017_forcings.idlsave'
temps_file = '../data/raw/obs_temp_data/combined_temps_Jan_2019.csv'
models_file = '../data/raw/model_temp_data/Model timeseries.xlsx'

interim_path = "../data/interim/" # save interim data here

In [13]:
#Import forcings and temperatures. Replace month temperatures with annual means.

forcings = spio.readsav(forcing_file, python_dict =True, verbose=False)
temps = pd.read_csv(temps_file)
annual_temps = temps.groupby('year').mean()
annual_temps = annual_temps.reset_index()[['year', 'hadcrut4', 'gistemp', 'noaa', 'berkeley', 'cowtan_way']]
single_models = pd.read_excel(models_file, sheet_name = 'Individual papers')
single_models.columns = map(str.lower, single_models.columns)

timeframes = ([1970, 2000], [1971, 2000], [1972, 2000], [1975, 2010], [1977, 2017], 
              [1981, 2017], [1988, 2017], [1990, 2017], [1993, 2017], [1995, 2017], 
              [2001, 2017], [2007, 2017])
model_names = ['manabe_1970', 'mitchell_1970', 'benson_1970', 'rasool_schneider_1971', 'sawyer_1972', 
               'broecker_1975', 'nordhaus_1977', 'schneider_thompson_1981', 'hansen_1981_1', 'hansen_1981_2a', 
               'hansen_1988_a', 'hansen_1988_b', 'hansen_1988_c', 'manabe_stouffer_1993', 'far_ebm_best', 
               'far_ebm_high', 'far_ebm_low', 'sar_ebm_best', 'sar_ebm_high', 'sar_ebm_low', 'tar_ebm_best', 
               'tar_ebm_high', 'tar_ebm_low', 'ar4_mmm_best', 'ar4_mmm_high', 'ar4_mmm_low']

model_years = ([1970, 2000], [1970, 2000], [1970, 2000], [1971, 2000], [1972, 2000], [1975, 2010], [1977, 2017], 
              [1981, 2017], [1981, 2017], [1981, 2017], [1988, 2017], [1988, 2017], [1988, 2017], [1993, 2017],
              [1990, 2017], [1990, 2017], [1990, 2017], [1995, 2017], [1995, 2017], [1995, 2017], [2001, 2017], 
              [2001, 2017], [2001, 2017], [2007, 2017], [2007, 2017], [2007, 2017])

ipcc_models = ['FAR EMB', 'FAR GFDL', 'FAR UKMET', 'SAR EMB', 'SAR MP01GG01', 'SAR CS01GG01', 'SAR GF01GG01', 
               'SAR DK01GG01', 'SAR NC01GG01', 'SAR HC02GG01', 'SAR HC01GG01', 'SAR HC01GG02', 'SAR HC01GG03', 
               'SAR HC01GG04', 'SAR NI01GG01', 'SAR CC01GG01', 'TAR EBM', 'TAR EH4OPYC_A2', 'TAR HADCM3_A2', 
               'TAR CSIRO_A2', 'TAR NCAR-CSM_A2', 'TAR CGCM2_A2', 'TAR CCSRNIES_A2']

ipcc_years = ([1990, 2017], [1990, 2017], [1990, 2017], [1995, 2017], [1995, 2017], [1995, 2017], [1995, 2017], 
              [1995, 2017], [1995, 2017], [1995, 2017], [1995, 2017], [1995, 2017], [1995, 2017], [1995, 2017], 
              [1995, 2017], [1995, 2017], [2001, 2017], [2001, 2017], [2001, 2017], [2001, 2017], [2001, 2017], 
              [2001, 2017], [2001, 2017])

/anaconda3/envs/OldModels/lib/python3.8/site-packages/xlrd/xlsx.py:266: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():


In [15]:
def coef_arma_cis(y_data, x_data, runtype='ar1'):
    '''
    Calculate coefficients using OLS and CIs via AR[1]
    '''
    X = x_data
    X = sm.add_constant(X)
    smresults = sm.OLS(y_data, X).fit()
    ols_coef = smresults.params[1]
    model = ARIMA(y_data, exog=x_data, order=(1,0,0)).fit(disp=0)
    arma_coef = model.params[1]
    
    ols_ci = ols_coef - smresults.conf_int(alpha=0.05, cols=None)[0][1]
    arma_ci = arma_coef - model.conf_int(alpha=0.05, cols=None)[0][1]
    if arma_ci > ols_ci:
        ci_lower = model.conf_int(alpha=0.05, cols=None)[0][1] - arma_coef + ols_coef
        ci_upper = model.conf_int(alpha=0.05, cols=None)[1][1] - arma_coef + ols_coef
    else:
        ci_lower = smresults.conf_int(alpha=0.05, cols=None)[0][1]
        ci_upper = smresults.conf_int(alpha=0.05, cols=None)[1][1]
    if runtype == 'ols':
        ci_lower = smresults.conf_int(alpha=0.05, cols=None)[0][1]
        ci_upper = smresults.conf_int(alpha=0.05, cols=None)[1][1]
    sd = (ci_upper - ols_coef) / 2.
    return {
            'coef' : ols_coef,
            'ci_lower' : ci_lower,
            'ci_upper' : ci_upper,
            'sd' : sd
    }


def obs_temp_time_trends(annual_temps, timeframes):
    coef_mean, coef_sd, ci_mean, timeframe = [], [], [], []
    for times in timeframes:
        print('Analyzing the period from ', times[0], ' to ', times[1])
        coef, ci_lower, ci_upper, rf_number, obs_series = [], [], [], [], []
        start_year = times[0]
        end_year = times[1]
        temp_year_range = np.where((annual_temps['year'] >= start_year) & (annual_temps['year'] <= end_year))[0]

        for obs_temps in (['hadcrut4', 'gistemp', 'noaa', 'berkeley', 'cowtan_way']):
            years = annual_temps['year'][temp_year_range]
            temp_values = annual_temps[obs_temps][temp_year_range]
            results = coef_arma_cis(temp_values, years)
            coef.append(results['coef'])
            ci_lower.append(results['ci_lower'])
            ci_upper.append(results['ci_upper'])
            obs_series.append(obs_temps)
        df = pd.DataFrame({'coef' : coef,
                           'ci_lower' : ci_lower,
                           'ci_upper' : ci_upper,
                           'obs_series' : obs_series})
        df['ci_val'] = df['coef'] - df['ci_lower']
        coef_mean.append(df['coef'].mean())
        coef_sd.append(df['coef'].std())
        ci_mean.append(df['ci_val'].mean())
        timeframe.append(str(times))
    
    df = pd.DataFrame({'coef_mean' : coef_mean,
                       'coef_sd' : coef_sd,
                       'ci_mean' : ci_mean,
                       'timeframe' : timeframe})
    uncertainty = ((df['coef_sd']*2)**2 + df['ci_mean']**2)**(0.5)
    df['coef_low'] = df['coef_mean'] - uncertainty
    df['coef_high'] = df['coef_mean'] + uncertainty
    df.to_csv(interim_path+'obs_time_trends.csv')


def obs_temp_forcing_trends(forcings, annual_temps, timeframes):
    coef_mean, coef_sd, ci_mean, timeframe = [], [], [], []

    for times in timeframes:
        print('Analyzing the period from ', times[0], ' to ', times[1])
        coef, ci_lower, ci_upper, rf_number, obs_series = [], [], [], [], []
        start_year = times[0]
        end_year = times[1]
        rf_anthro = np.swapaxes(forcings['rf_anthro'],0,1)   
        rf_year_range = np.where((forcings['year'] >= start_year) & (forcings['year'] <= end_year))[0]
        temp_year_range = np.where((annual_temps['year'] >= start_year) & (annual_temps['year'] <= end_year))[0]
        
        for rf_num in range(1000):
            for obs_temps in (['hadcrut4', 'gistemp', 'noaa', 'berkeley', 'cowtan_way']):
                rf_values = rf_anthro[rf_num][rf_year_range]
                temp_values = annual_temps[obs_temps][temp_year_range]
                results = coef_arma_cis(temp_values, rf_values, runtype='ols')
                coef.append(results['coef'])
                ci_lower.append(results['ci_lower'])
                ci_upper.append(results['ci_upper'])
                rf_number.append(rf_num)
                obs_series.append(obs_temps)
        df = pd.DataFrame({'coef' : coef,
                           'ci_lower' : ci_lower,
                           'ci_upper' : ci_upper,
                           'rf_number' : rf_number,
                           'obs_series' : obs_series})
        df['ci_val'] = df['coef'] - df['ci_lower']
        coef_mean.append(df['coef'].mean())
        coef_sd.append(df['coef'].std())
        ci_mean.append(df['ci_val'].mean())
        timeframe.append(str(times))
    
    df = pd.DataFrame({'coef_mean' : coef_mean,
                       'coef_sd' : coef_sd,
                       'ci_mean' : ci_mean,
                       'timeframe' : timeframe})
    uncertainty = ((df['coef_sd']*2)**2 + df['ci_mean']**2)**(0.5)
    df['coef_low'] = df['coef_mean'] - uncertainty
    df['coef_high'] = df['coef_mean'] + uncertainty
    df.to_csv(interim_path+'obs_forcing_trends.csv')


def models_forcing_time_trends(single_models, model_names, model_years):
    coef, coef_low, coef_high, timeframe, model, dtype = [], [], [], [], [], []
    for i in range(len(model_names)):
        print('Analyzing '+model_names[i]+' from '+str(model_years[i][0])+' to '+str(model_years[i][1]))
        years = single_models['year'].between(model_years[i][0], model_years[i][1])
        temp_year = coef_arma_cis(single_models[model_names[i]+'_t'][years], single_models['year'][years])
        temp_forcing = coef_arma_cis(single_models[model_names[i]+'_t'][years], single_models[model_names[i]+'_f'][years], runtype='ols')
        coef.append(temp_year['coef'])
        coef_low.append(temp_year['ci_lower'])
        coef_high.append(temp_year['ci_upper'])
        timeframe.append(str(model_years[i][0])+' to '+str(model_years[i][1]))
        model.append(model_names[i])
        dtype.append('model_time')
        coef.append(temp_forcing['coef'])
        coef_low.append(temp_forcing['ci_lower'])
        coef_high.append(temp_forcing['ci_upper'])
        timeframe.append(str(model_years[i][0])+' to '+str(model_years[i][1]))
        model.append(model_names[i])
        dtype.append('model_forcing')
    #manabe_1970_f = coef_arma_cis(df['manabe_1970_t'], df['manabe_1970_f'])
    
    df = pd.DataFrame({'coef' : coef,
                       'coef_low' : coef_low,
                       'coef_high' : coef_high,
                       'timeframe' : timeframe,
                       'model' : model,
                       'dtype': dtype})
    df.to_csv(interim_path+'single_model_trends.csv')


def model_obs_time_diffs(single_models, model_names, model_years, annual_temps):
    coef_mean, coef_sd, ci_mean, timeframe, model = [], [], [], [], []
    for i in range(len(model_names)):
        coef, ci_lower, ci_upper = [], [], []
        years = single_models['year'].between(model_years[i][0], model_years[i][1])
        for obs_temps in (['hadcrut4', 'gistemp', 'noaa', 'berkeley', 'cowtan_way']):
            print('Analyzing '+model_names[i]+' '+obs_temps+' diffs from '+str(model_years[i][0])+' to '+str(model_years[i][1]))
            model_obs_diff = single_models[model_names[i]+'_t'][years] - annual_temps[obs_temps][years]            
            results = coef_arma_cis(model_obs_diff, single_models['year'][years])
            coef.append(results['coef'])
            ci_lower.append(results['ci_lower'])
            ci_upper.append(results['ci_upper'])
        df = pd.DataFrame({'coef' : coef,
                           'ci_lower' : ci_lower,
                           'ci_upper' : ci_upper})
        df['ci_val'] = df['coef'] - df['ci_lower']
        coef_mean.append(df['coef'].mean())
        coef_sd.append(df['coef'].std())
        ci_mean.append(df['ci_val'].mean())
        timeframe.append(str(model_years[i]))
        model.append(str(model_names[i]))
    
    df = pd.DataFrame({'coef_mean' : coef_mean,
                       'coef_sd' : coef_sd,
                       'ci_mean' : ci_mean,
                       'model' : model,
                       'timeframe' : timeframe})
    uncertainty = ((df['coef_sd']*2)**2 + df['ci_mean']**2)**(0.5)
    df['coef_low'] = df['coef_mean'] - uncertainty
    df['coef_high'] = df['coef_mean'] + uncertainty
    df.to_csv(interim_path+'model_obs_time_diffs.csv')    

obs_temp_time_trends(annual_temps, timeframes)
obs_temp_forcing_trends(forcings, annual_temps, timeframes)
models_forcing_time_trends(single_models, model_names, model_years)
model_obs_time_diffs(single_models, model_names, model_years, annual_temps)

Analyzing the period from  1970  to  2000


/anaconda3/envs/OldModels/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/anaconda3/envs/OldModels/lib/python3.8/site-packages/statsmodels/base/model.py:511: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "


Analyzing the period from  1971  to  2000
Analyzing the period from  1972  to  2000
Analyzing the period from  1975  to  2010
Analyzing the period from  1977  to  2017
Analyzing the period from  1981  to  2017
Analyzing the period from  1988  to  2017
Analyzing the period from  1990  to  2017
Analyzing the period from  1993  to  2017
Analyzing the period from  1995  to  2017
Analyzing the period from  2001  to  2017
Analyzing the period from  2007  to  2017
Analyzing the period from  1970  to  2000
Analyzing the period from  1971  to  2000
Analyzing the period from  1972  to  2000


KeyboardInterrupt: 

In [6]:
def output_forcing_ensemble(forcings, annual_temps, start_year, anom_period, path="./"):
    rf_year_range = np.where((forcings['year'] >= start_year) & (forcings['year'] <= 2017))[0]
    temp_year_range = np.where((annual_temps['year'] >= start_year) & (annual_temps['year'] <= 2017))[0]
    rf_anthro = forcings['rf_anthro'][rf_year_range]
    rf_anthro_anoms = rf_anthro - rf_anthro[0:anom_period].mean(axis=0)
    rf_anthro_anoms = np.swapaxes(rf_anthro_anoms,0,1)

    df_names = ('hadcrut4_df', 'gistemp_df', 'noaa_df', 'berkeley_df', 'cowtan_way_df')
    dfs ={}

    for df_names,obs_temps in zip(df_names, ['hadcrut4', 'gistemp', 'noaa', 'berkeley', 'cowtan_way']):
        dfs[df_names] = pd.DataFrame()
        dfs[df_names]['year'] = annual_temps['year'][temp_year_range]
        temps = annual_temps[obs_temps][temp_year_range].values
        anoms = temps - temps[0:anom_period].mean(axis=0)
        dfs[df_names]['temp'] = anoms
        for rf_num in range(1000):
            dfs[df_names]['forcing_'+str(rf_num)] = rf_anthro_anoms[rf_num]
        
        dfs[df_names].to_csv(path+obs_temps + '_' + 'forcings'+str(start_year)+'.csv')

output_forcing_ensemble(forcings, annual_temps, 1970, 20, path="../data/processed/FAR_spaghetti/")
output_forcing_ensemble(forcings, annual_temps, 1988, 20, path="../data/processed/Hansen_spaghetti/")

In [7]:
def model_forcing_rate(single_models, model_names, model_years):
    coef, coef_low, coef_high, timeframe, model = [], [], [], [], []
    for i in range(len(model_names)):
        print('Analyzing '+model_names[i]+' from '+str(model_years[i][0])+' to '+str(model_years[i][1]))
        years = single_models['year'].between(model_years[i][0], model_years[i][1])
        forcing_rate = coef_arma_cis(single_models[model_names[i]+'_f'][years], single_models['year'][years])
        coef.append(forcing_rate['coef'])
        coef_low.append(forcing_rate['ci_lower'])
        coef_high.append(forcing_rate['ci_upper'])
        timeframe.append(str(model_years[i][0])+' to '+str(model_years[i][1]))
        model.append(model_names[i])
    #manabe_1970_f = coef_arma_cis(df['manabe_1970_t'], df['manabe_1970_f'])
    
    df = pd.DataFrame({'coef' : coef,
                       'coef_low' : coef_low,
                       'coef_high' : coef_high,
                       'timeframe' : timeframe,
                       'model' : model})
    df.to_csv(interim_path+'single_model_forcing_rate.csv')

def obs_forcing_rate(forcings, annual_temps, timeframes):
    coef_mean, coef_sd, ci_mean, timeframe = [], [], [], []
    
    for times in timeframes:
        print('Analyzing the period from ', times[0], ' to ', times[1])
        coef, ci_lower, ci_upper, rf_number, obs_series = [], [], [], [], []
        start_year = times[0]
        end_year = times[1]
        rf_anthro = np.swapaxes(forcings['rf_anthro'],0,1)   
        rf_year_range = np.where((forcings['year'] >= start_year) & (forcings['year'] <= end_year))[0]
        temp_year_range = np.where((annual_temps['year'] >= start_year) & (annual_temps['year'] <= end_year))[0]
        
        for rf_num in range(1000):
            for obs_temps in (['hadcrut4', 'gistemp', 'noaa', 'berkeley', 'cowtan_way']):
                rf_values = rf_anthro[rf_num][rf_year_range]
                years = annual_temps['year'][temp_year_range]
                results = coef_arma_cis(rf_values, years)
                coef.append(results['coef'])
                ci_lower.append(results['ci_lower'])
                ci_upper.append(results['ci_upper'])
                rf_number.append(rf_num)
                obs_series.append(obs_temps)
        df = pd.DataFrame({'coef' : coef,
                           'ci_lower' : ci_lower,
                           'ci_upper' : ci_upper,
                           'rf_number' : rf_number,
                           'obs_series' : obs_series})
        df['ci_val'] = df['coef'] - df['ci_lower']
        coef_mean.append(df['coef'].mean())
        coef_sd.append(df['coef'].std())
        ci_mean.append(df['ci_val'].mean())
        timeframe.append(str(times))
    
    df = pd.DataFrame({'coef_mean' : coef_mean,
                       'coef_sd' : coef_sd,
                       'ci_mean' : ci_mean,
                       'timeframe' : timeframe})
    uncertainty = ((df['coef_sd']*2)**2 + df['ci_mean']**2)**(0.5)
    df['coef_low'] = df['coef_mean'] - uncertainty
    df['coef_high'] = df['coef_mean'] + uncertainty
    df.to_csv(interim_path+'obs_forcing_rate.csv')

model_forcing_rate(single_models, model_names, model_years)
obs_forcing_rate(forcings, annual_temps, timeframes)

Analyzing the period from  1970  to  2000


/usr/local/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:606: RuntimeWarning: overflow encountered in exp
  newparams = ((1-np.exp(-params))/
/usr/local/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:607: RuntimeWarning: invalid value encountered in true_divide
  (1+np.exp(-params))).copy()


Analyzing the period from  1971  to  2000
Analyzing the period from  1972  to  2000
Analyzing the period from  1975  to  2010


/usr/local/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:634: RuntimeWarning: divide by zero encountered in log
  invarcoefs = -np.log((1-params)/(1+params))
/usr/local/lib/python3.7/site-packages/statsmodels/tsa/kalmanf/kalmanfilter.py:654: RuntimeWarning: divide by zero encountered in log
  R_mat, T_mat)
/usr/local/lib/python3.7/site-packages/statsmodels/tools/numdiff.py:243: RuntimeWarning: invalid value encountered in subtract
  **kwargs)).imag/2./hess[i, j]
/usr/local/lib/python3.7/site-packages/statsmodels/tools/numdiff.py:243: RuntimeWarning: invalid value encountered in multiply
  **kwargs)).imag/2./hess[i, j]


Analyzing the period from  1977  to  2017
Analyzing the period from  1981  to  2017
Analyzing the period from  1988  to  2017
Analyzing the period from  1990  to  2017
Analyzing the period from  1993  to  2017
Analyzing the period from  1995  to  2017
Analyzing the period from  2001  to  2017
Analyzing the period from  2007  to  2017


In [20]:
def obs_forcing_timeseries(forcings):
    forcings['rf_total'].shape
    std = np.std(forcings['rf_total'], axis=1)
    mean = np.mean(forcings['rf_total'], axis=1)
    
    df = pd.DataFrame({'year' : forcings['year'],
                       'forcing_mean' : mean,
                       'forcing_std' : std})
    df.to_csv(interim_path+'forcing_timeseries.csv')

obs_forcing_timeseries(forcings)

In [38]:
def model_skill(obs_trend, pred_trend):
    skill = 1 - ((obs_trend - pred_trend)**2 / obs_trend**2) ** (0.5)
    return skill


def model_time_skill_scores(single_models, model_names, model_years, annual_temps):
    skill_mean, skill_median, skill_5th, skill_95th, timeframe, model = [], [], [], [], [], []
    for i in range(len(model_names)):
        skill = []
        years = single_models['year'].between(model_years[i][0], model_years[i][1])
        model_trend = coef_arma_cis(single_models[model_names[i]+'_t'][years], single_models['year'][years])
        for obs_temps in (['hadcrut4', 'gistemp', 'noaa', 'berkeley', 'cowtan_way']):
            print('Analyzing '+model_names[i]+' '+obs_temps+' diffs from '+str(model_years[i][0])+' to '+str(model_years[i][1]))
            obs_trend = coef_arma_cis(annual_temps[obs_temps][years], annual_temps['year'][years])
            model_monte_carlo = np.random.normal(model_trend['coef'], model_trend['sd'], 100)
            obs_monte_carlo = np.random.normal(obs_trend['coef'], obs_trend['sd'], 100)
            for j in range(100):
                skill.append(model_skill(obs_monte_carlo[j], model_monte_carlo[j]))
        df = pd.DataFrame({'skill' : skill})
        skill_mean.append(df['skill'].mean())
        skill_median.append(df['skill'].median())
        skill_5th.append(df['skill'].quantile(0.05))
        skill_95th.append(df['skill'].quantile(0.95))    
        timeframe.append(str(model_years[i]))
        model.append(str(model_names[i]))
    
    df = pd.DataFrame({'skill_mean' : skill_mean,
                       'skill_median' : skill_median,
                       'skill_5th' : skill_5th,
                       'skill_95th' : skill_95th,
                       'model' : model,
                       'timeframe' : timeframe})
    df.to_csv(interim_path+'model_time_skill_scores.csv')
    

def model_tcr_skill_scores(single_models, model_names, model_years, annual_temps, forcings):
    skill_mean, skill_median, skill_5th, skill_95th, timeframe, model = [], [], [], [], [], []
    rf_anthro = np.swapaxes(forcings['rf_anthro'],0,1)   
    for i in range(len(model_names)):
        print('Analyzing '+model_names[i]+' from '+str(model_years[i][0])+' to '+str(model_years[i][1]))
        skill = []
        years = single_models['year'].between(model_years[i][0], model_years[i][1])
        model_t = single_models[model_names[i]+'_t'][years]
        model_f = single_models[model_names[i]+'_f'][years]
        model_tcr = coef_arma_cis(model_t, model_f, runtype='ols')
        rf_year_range = np.where((forcings['year'] >= model_years[i][0]) & (forcings['year'] <= model_years[i][1]))[0]
        for rf_num in range(1000):
            for obs_temps in (['hadcrut4', 'gistemp', 'noaa', 'berkeley', 'cowtan_way']):
                obs_f = rf_anthro[rf_num][rf_year_range]
                obs_t = annual_temps[obs_temps][years]
                obs_tcr = coef_arma_cis(obs_t, obs_f, runtype='ols')
                model_monte_carlo = np.random.normal(model_tcr['coef'], model_tcr['sd'], 100)
                obs_monte_carlo = np.random.normal(obs_tcr['coef'], obs_tcr['sd'], 100)
                for j in range(100):
                    skill.append(model_skill(obs_monte_carlo[j], model_monte_carlo[j]))
        df = pd.DataFrame({'skill' : skill})
        skill_mean.append(df['skill'].mean())
        skill_median.append(df['skill'].median())        
        skill_5th.append(df['skill'].quantile(0.05))
        skill_95th.append(df['skill'].quantile(0.95))        
        timeframe.append(str(model_years[i]))
        model.append(str(model_names[i]))
    
    df = pd.DataFrame({'skill_mean' : skill_mean,
                       'skill_median' : skill_median,
                       'skill_5th' : skill_5th,
                       'skill_95th' : skill_95th,
                       'model' : model,
                       'timeframe' : timeframe})
    df.to_csv(interim_path+'model_tcr_skill_scores.csv')    
    
    

model_time_skill_scores(single_models, model_names, model_years, annual_temps)
model_tcr_skill_scores(single_models, model_names, model_years, annual_temps, forcings)
    

Analyzing manabe_1970 from 1970 to 2000
Analyzing mitchell_1970 from 1970 to 2000
Analyzing benson_1970 from 1970 to 2000
Analyzing rasool_schneider_1971 from 1971 to 2000
Analyzing sawyer_1972 from 1972 to 2000
Analyzing broecker_1975 from 1975 to 2010
Analyzing nordhaus_1977 from 1977 to 2017
Analyzing schneider_thompson_1981 from 1981 to 2017
Analyzing hansen_1981_1 from 1981 to 2017
Analyzing hansen_1981_2a from 1981 to 2017
Analyzing hansen_1988_a from 1988 to 2017
Analyzing hansen_1988_b from 1988 to 2017
Analyzing hansen_1988_c from 1988 to 2017
Analyzing manabe_stouffer_1993 from 1993 to 2017
Analyzing far_ebm_best from 1990 to 2017
Analyzing far_ebm_high from 1990 to 2017
Analyzing far_ebm_low from 1990 to 2017
Analyzing sar_ebm_best from 1995 to 2017
Analyzing sar_ebm_high from 1995 to 2017
Analyzing sar_ebm_low from 1995 to 2017
Analyzing tar_ebm_best from 2001 to 2017
Analyzing tar_ebm_high from 2001 to 2017
Analyzing tar_ebm_low from 2001 to 2017
Analyzing ar4_mmm_best fro

In [ ]:


def model_obs_forcing_diffs(single_models, model_names, model_years, forcings, annual_temps):
    coef_mean, coef_sd, ci_mean, timeframe, model = [], [], [], [], []
    for i in range(len(model_names)):
        print('Analyzing '+model_names[i]+' diffs from '+str(model_years[i][0])+' to '+str(model_years[i][1]))
        coef, ci_lower, ci_upper, rf_number, obs_series = [], [], [], [], []
        model_year_range = single_models['year'].between(model_years[i][0], model_years[i][1])
        rf_anthro = np.swapaxes(forcings['rf_anthro'],0,1)   
        rf_year_range = np.where((forcings['year'] >= model_years[i][0]) & (forcings['year'] <= model_years[i][1]))[0]
        temp_year_range = np.where((annual_temps['year'] >= model_years[i][0]) & (annual_temps['year'] <= model_years[i][1]))[0]
        model_temp = single_models[model_names[i]+'_t'][model_year_range]
        model_rf = single_models[model_names[i]+'_f'][model_year_range]

        for rf_num in range(1000):
            for obs_temps in (['hadcrut4', 'gistemp', 'noaa', 'berkeley', 'cowtan_way']):
                obs_rf = rf_anthro[rf_num][rf_year_range]
                obs_temp = annual_temps[obs_temps][temp_year_range]
                temp_diff = model_temp - obs_temp
                rf_diff = model_rf - obs_rf
                results = coef_arma_cis(temp_diff, rf_diff, runtype='ols')
                coef.append(results['coef'])
                ci_lower.append(results['ci_lower'])
                ci_upper.append(results['ci_upper'])
                rf_number.append(rf_num)
                obs_series.append(obs_temps)
        df = pd.DataFrame({'coef' : coef,
                           'ci_lower' : ci_lower,
                           'ci_upper' : ci_upper,
                           'rf_number' : rf_number,
                           'obs_series' : obs_series})
        df['ci_val'] = df['coef'] - df['ci_lower']
        coef_mean.append(df['coef'].mean())
        coef_sd.append(df['coef'].std())
        ci_mean.append(df['ci_val'].mean())
        timeframe.append(str(model_years[i]))
        model.append(str(model_names[i]))    
    df = pd.DataFrame({'coef_mean' : coef_mean,
                       'coef_sd' : coef_sd,
                       'ci_mean' : ci_mean,
                       'model' : model,
                       'timeframe' : timeframe})
    uncertainty = ((df['coef_sd']*2)**2 + df['ci_mean']**2)**(0.5)
    df['coef_low'] = df['coef_mean'] - uncertainty
    df['coef_high'] = df['coef_mean'] + uncertainty
    df.to_csv(interim_path+'model_obs_forcing_trend_diffs.csv')

model_obs_forcing_diffs(single_models, model_names, model_years, forcings, annual_temps)
